In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#shapefile_parcels = gpd.read_file('zip:///Users/jameskreft/AnacondaProjects/madison_parcels_data/Tax_Parcels_shapefile_2022_02_15.zip')


In [ ]:
#shapefile_parcels.info(verbose=True)

You can get the parcels data from the [Madison Open data portal](https://data-cityofmadison.opendata.arcgis.com/datasets/0338b0638e4749c395f8d38b39a5c466_0/explore), choose download, and then choose either geojson or file geodatabase.  Both kick out the same data that geopandas reads. Don't use shapefile, it truncates the data headers and makes everything confusing.

In [ ]:
#parcels = gpd.read_file('Tax_Parcels_March_2024.geojson')
parcels = gpd.read_file('parcels_filegeodatabase.gdb')


In [ ]:
parcels.info(verbose=True)

This is an attempt to clean up the data to colsolidate the different property uses.  It needs more work.

In [ ]:


def clean_up_property_use(prop_use):
    if 'apartment' in prop_use.lower():
        cleaned_use = 'Multifamily'
    elif (prop_use == '2 Unit' or prop_use == '3 To 7 Unit'):
        cleaned_use = 'Multifamily'
    elif (prop_use == 'Condominium -apt'):
        cleaned_use = 'Multifamily'
    elif prop_use.startswith('Warehouse') or prop_use.startswith('Warehse'):
        cleaned_use = 'Warehouse'
    elif prop_use.startswith('Store'):
        cleaned_use = 'Store'
    elif prop_use.startswith('Station'):
        cleaned_use = 'Station'
    else:
        cleaned_use = prop_use
    return(cleaned_use)

parcels['property_use_cleaned'] = parcels['PropertyUse'].apply(clean_up_property_use)
#parcels.head(50)
multifamily = parcels[parcels['property_use_cleaned']=='Multifamily']
#multifamily.info(verbose=True)
multifamily_grouped = multifamily.groupby(["MaxConstructionYear"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"})
multifamily_grouped['construction_by_decade'] = pd.cut(x=multifamily_grouped['MaxConstructionYear'], bins=[1700,1899,1909,1919,1929,1939,1949,1959,1969,1979,1989,1999,2009,2019,2029], 
                                                       labels=['1800-1899', '1900s','1910s', '1920s', '1930s', '1940s', '1950s', '1960s','1970s','1980s', '1990s','2000s','2010s','2020s'])

multifamily_grouped.head(50)

ax = multifamily_grouped.groupby(["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"}).plot.bar(x='construction_by_decade', y='TotalDwellingUnits', rot=0, ylabel='Total Dwelling units')

plt.xticks(rotation=45)
plt.show()



In [ ]:
narrowed_multifamily = multifamily.loc[:,['OBJECTID', 'Parcel', 'XRefParcel', 'Address', 'PropertyClass', 'PropertyUse','property_use_cleaned',
                                  'AreaName', 'Bedrooms', 'CurrentTotal', 'NetTaxes', 'TotalTaxes', 'LotSize', 
                                  'Zoning1', 'LotDepth', 'LotWidth', 'LotType1', 'LotType2','Ward', 'AlderDistrict','NeighborhoodPrimary', 'TotalDwellingUnits','geometry']]
narrowed_multifamily.explore()

In [ ]:
multifamily_zero_dwelling_units = narrowed_multifamily[narrowed_multifamily['TotalDwellingUnits']==0]
multifamily_zero_dwelling_units.info()

In [ ]:
multifamily_zero_dwelling_units.explore()

In [ ]:
alder_2_4 = multifamily[multifamily.AlderDistrict.isin([2,4])]
alder_2_4_multifamily_grouped = alder_2_4.groupby(["MaxConstructionYear"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"})
alder_2_4_multifamily_grouped['construction_by_decade'] = pd.cut(x=alder_2_4_multifamily_grouped['MaxConstructionYear'], bins=[1700,1899,1909,1919,1929,1939,1949,1959,1969,1979,1989,1999,2009,2019,2029], 
                                                       labels=['1800-1899', '1900s','1910s', '1920s', '1930s', '1940s', '1950s', '1960s','1970s','1980s', '1990s','2000s','2010s','2020s'])


ax = alder_2_4_multifamily_grouped.groupby(["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"}).plot.bar(x='construction_by_decade', y='TotalDwellingUnits', rot=0, ylabel='Total Dwelling units')

plt.xticks(rotation=45)
plt.show()

In [ ]:
alder_6 = multifamily[multifamily.AlderDistrict.isin([6])]
alder_6_multifamily_grouped = alder_6.groupby(["MaxConstructionYear"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"})
alder_6_multifamily_grouped['construction_by_decade'] = pd.cut(x=alder_6_multifamily_grouped['MaxConstructionYear'], bins=[1700,1899,1909,1919,1929,1939,1949,1959,1969,1979,1989,1999,2009,2019,2029], 
                                                       labels=['1800-1899', '1900s','1910s', '1920s', '1930s', '1940s', '1950s', '1960s','1970s','1980s', '1990s','2000s','2010s','2020s'])

#print(alder_6_multifamily_grouped.groupby(["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"}).to_markdown())

ax = alder_6_multifamily_grouped.groupby(["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"}).plot.bar(x='construction_by_decade', y='TotalDwellingUnits', rot=0, 
                                                                                                                                           ylabel='Total MultiFamily Dwelling units', xlabel='Decade of Construction')
plt.title("Units of multifamily housing built in Madison Alder District 6 per decade")
plt.xticks(rotation=45)
plt.show()

In [ ]:
alder_district = 8

alder_df = multifamily[multifamily.AlderDistrict == alder_district]
alder_df_multifamily_grouped = alder_df.groupby(["MaxConstructionYear"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"})
alder_df_multifamily_grouped['construction_by_decade'] = pd.cut(x=alder_df_multifamily_grouped['MaxConstructionYear'], bins=[1700,1899,1909,1919,1929,1939,1949,1959,1969,1979,1989,1999,2009,2019,2029], 
                                                       labels=['1800-1899', '1900s','1910s', '1920s', '1930s', '1940s', '1950s', '1960s','1970s','1980s', '1990s','2000s','2010s','2020s'])

#print(alder_6_multifamily_grouped.groupby(["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"}).to_markdown())

alder_df_max_const_decade = alder_df_multifamily_grouped.groupby(
    ["construction_by_decade"], sort=True, as_index=False).agg({"TotalDwellingUnits":"sum"})

ax = alder_df_max_const_decade.plot.bar(x='construction_by_decade', y='TotalDwellingUnits', rot=0, 
                                                                                                                                           ylabel='Total MultiFamily Dwelling units', xlabel='Decade of Construction')
plt.title(f"Units of multifamily housing built in Madison Alder District {alder_district} per decade")
plt.xticks(rotation=45)
plt.show()


In [ ]:
narrowed_parcels = parcels.loc[:,['Parcel', 'XRefParcel', 'Address', 'PropertyClass', 'PropertyUse',
                                  'AreaName', 'Bedrooms', 'CurrentTotal', 'NetTaxes', 'TotalTaxes', 'LotSize', 'SHAPE_Area',
                                  'Zoning1', 'LotDepth', 'LotWidth', 'LotType1', 'LotType2','Ward', 'AlderDistrict','NeighborhoodPrimary', 'TotalDwellingUnits','geometry']]


In [ ]:
narrowed_parcels.explore()

In [ ]:
narrowed_parcels["MostCommonPropertyClass"] = (narrowed_parcels.groupby("XRefParcel")["PropertyClass"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonPropertyUse"] = (narrowed_parcels.groupby("XRefParcel")["PropertyUse"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonAreaName"] = (narrowed_parcels.groupby("XRefParcel")["AreaName"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonLotType1"] = (narrowed_parcels.groupby("XRefParcel")["LotType1"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonLotType2"] = (narrowed_parcels.groupby("XRefParcel")["LotType2"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonZoning1"] = (narrowed_parcels.groupby("XRefParcel")["Zoning1"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonNeighborhoodPrimary"] = (narrowed_parcels.groupby("XRefParcel")["NeighborhoodPrimary"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels_grouped = narrowed_parcels.groupby(["XRefParcel","geometry", "MostCommonPropertyClass", 
                  "MostCommonPropertyUse", "MostCommonAreaName", "MostCommonLotType1", "MostCommonLotType2", 
                  "MostCommonZoning1", "Ward", 'AlderDistrict','MostCommonNeighborhoodPrimary'], sort=False, as_index=False).agg(
    {"Address":"min", "Bedrooms":"sum", "CurrentTotal":"sum", "NetTaxes":"sum","TotalTaxes":"sum", 
     "LotSize":"max","SHAPE_Area":"max", "TotalDwellingUnits":"sum"})

narrowed_parcels_grouped['MostCommonPropertyUseCleaned'] = narrowed_parcels_grouped['MostCommonPropertyUse'].apply(clean_up_property_use)

narrowed_parcels_grouped = gpd.GeoDataFrame(narrowed_parcels_grouped, geometry='geometry')

#narrowed_parcels_grouped.explore("taxes_per_sq_foot", cmap="Blues",tooltip=False, popup=True, tiles='Stamen Toner' )

In [ ]:
bad_lot_size = narrowed_parcels_grouped.loc[narrowed_parcels_grouped['LotSize'].isin([0.00, 0.01, 1.00, 6.00])]
bad_lot_size.explore()

In [ ]:
#set to a proper equal area projection, and calculate a square foot
bad_lot_size = bad_lot_size.to_crs("EPSG:5070")
bad_lot_size["SqFeet"] = bad_lot_size.area*10.7639
bad_lot_size.head(24)

In [ ]:
for index, row in bad_lot_size.iterrows():
    print(index, row['XRefParcel'],row['SqFeet'])
    print(narrowed_parcels_grouped.at[index,'XRefParcel'], narrowed_parcels_grouped.at[index,'LotSize'])
    narrowed_parcels_grouped.at[index,'LotSize']=row['SqFeet']

In [ ]:
narrowed_parcels_grouped['taxes_per_sq_foot'] = narrowed_parcels_grouped['TotalTaxes']/narrowed_parcels_grouped['LotSize']

In [ ]:
taxes_top = narrowed_parcels_grouped.sort_values(by='taxes_per_sq_foot',ascending=False)
taxes_top.head(50)


In [ ]:
size = narrowed_parcels_grouped['taxes_per_sq_foot'].size-1
narrowed_parcels_grouped['percentile'] = narrowed_parcels_grouped['taxes_per_sq_foot'].rank(method='max').apply(lambda x: 100.0*(x-1)/size)
narrowed_parcels_grouped['taxes_per_acre'] = narrowed_parcels_grouped['taxes_per_sq_foot']*43560
narrowed_parcels_grouped.head(50)

In [ ]:
narrowed_parcels_grouped.hist(column='taxes_per_acre',  by='AlderDistrict', bins=25,)

In [ ]:
from datetime import date

today = date.today()

# dd/mm/YY
date = today.strftime("%Y-%m-%d")



narrowed_parcels_grouped.to_file(f"madison_taxes_per_sq_foot-{date}.geojson", driver='GeoJSON')

In [ ]:
alder_data = narrowed_parcels_grouped[['AlderDistrict', 'Bedrooms', 'CurrentTotal','NetTaxes', 'LotSize','TotalDwellingUnits','MostCommonZoning1']].groupby(['AlderDistrict']).sum()

alder_data.head(50)

In [ ]:
alder_data.to_pickle("alder_data.pkl")


You can get the neighborhood associations geojson from the [Madison Open Data Portal](https://data-cityofmadison.opendata.arcgis.com/datasets/66e4a6a80ae64865a81bc8d4464a6417_12/explore)

In [ ]:
nb_assoc = gpd.read_file('Neighborhood_Associations.geojson')
nb_assoc.head()

In [ ]:
nb_assoc.explore("ShapeSTArea", cmap="Blues") 

In [ ]:
bay_creek = nb_assoc[nb_assoc['NEIGHB_NAME']== 'Bay Creek Neighborhood Association']
bay_creek

In [ ]:
join_left_neigh_assoc_parcels = bay_creek.sjoin(narrowed_parcels_grouped, how="right")
join_left_neigh_assoc_parcels.explore()

In [ ]:
narrowed_parcels_grouped.head()
narrowed_parcels_grouped.info(verbose=True)

In [ ]:
res_difference = bay_creek.overlay(narrowed_parcels_grouped, how='intersection')
res_difference.explore()



In [ ]:
res_difference.info(verbose=True)

In [ ]:
bay_creek_narrowed = res_difference[['NA_ID','MostCommonPropertyUse','Bedrooms', 'CurrentTotal','NetTaxes', 'LotSize','TotalDwellingUnits']]


bay_creek_narrowed_grouped = bay_creek_narrowed.groupby(['NA_ID']).sum()

bay_creek_narrowed_grouped



In [ ]:
whole_city_nb_parcels = nb_assoc.overlay(narrowed_parcels_grouped, how='intersection')
whole_city_nb_parcels.explore()

In [ ]:
whole_city_nb_parcels_narrowed = whole_city_nb_parcels[['NA_ID','NEIGHB_NAME','Bedrooms', 'CurrentTotal','NetTaxes', 'LotSize','TotalDwellingUnits','MostCommonPropertyUse']]


whole_city_nb_parcels_narrowed_grouped = whole_city_nb_parcels_narrowed.groupby(['NA_ID','NEIGHB_NAME','MostCommonPropertyUse']).sum()

whole_city_nb_parcels_narrowed_grouped.head()

In [ ]:
whole_city_nb_parcels_narrowed_grouped.to_csv('neigh_associan_value_taxes.csv')